dont forget to update pipenv with mlflow and the other packages

batch deployment



In [1]:
import pandas as pd
import mlflow

In [2]:
categories = ["PULocationID", "DOLocationID"]
numerics = ["trip_distance"]
target = ["duration"]


def read_data(filename):
    df = pd.read_parquet(filename)
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df["duration"] = df["duration"].map(lambda x: x.total_seconds() / 60)

    df = df[(df["duration"] >= 0) & (df["duration"] <= 60)]
    df[categories] = df[categories].astype(object)

    df["tpep_pickup_datetime"] = df["tpep_pickup_datetime"].astype(int)

    prepped = df[categories + numerics + target].dropna()
    return prepped

In [17]:
import uuid

df = pd.read_parquet("data/yellow_tripdata_2024-04.parquet")
ride_id = []
for ride in range(len(df)):
    ride_id.append(str(uuid.uuid4()))

df["ride_id"] = ride_id
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,ride_id
0,1,2024-04-01 00:02:40,2024-04-01 00:30:42,0.0,5.20,1.0,N,161,7,1,29.6,3.5,0.5,8.65,0.0,1.0,43.25,2.5,0.0,f527a0d3-483d-422a-a0fd-253558622f88
1,2,2024-04-01 00:41:12,2024-04-01 00:55:29,1.0,5.60,1.0,N,264,264,1,25.4,1.0,0.5,10.00,0.0,1.0,37.90,0.0,0.0,8638a9ee-558d-413f-86a4-66e9c2a0f744
2,2,2024-04-01 00:48:42,2024-04-01 01:05:30,1.0,3.55,1.0,N,186,236,1,20.5,1.0,0.5,5.10,0.0,1.0,30.60,2.5,0.0,955357bd-7ce0-433f-916e-479de8b4c0f1
3,2,2024-04-01 00:56:02,2024-04-01 01:05:09,1.0,1.06,1.0,N,137,164,2,10.0,1.0,0.5,0.00,0.0,1.0,15.00,2.5,0.0,573376ae-b4e9-4c8d-baf5-1a9c10e9f888
4,1,2024-04-01 00:08:32,2024-04-01 00:10:24,1.0,0.70,1.0,N,236,263,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,775c1146-b91d-4efd-b423-0863185c2fa8


In [11]:
dicts = read_data("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet")[
    categories + numerics
].to_dict(orient="records")
dicts

[{'PULocationID': 161, 'DOLocationID': 7, 'trip_distance': 5.2},
 {'PULocationID': 264, 'DOLocationID': 264, 'trip_distance': 5.6},
 {'PULocationID': 186, 'DOLocationID': 236, 'trip_distance': 3.55},
 {'PULocationID': 137, 'DOLocationID': 164, 'trip_distance': 1.06},
 {'PULocationID': 236, 'DOLocationID': 263, 'trip_distance': 0.7},
 {'PULocationID': 142, 'DOLocationID': 68, 'trip_distance': 1.63},
 {'PULocationID': 138, 'DOLocationID': 146, 'trip_distance': 4.7},
 {'PULocationID': 186, 'DOLocationID': 87, 'trip_distance': 3.24},
 {'PULocationID': 132, 'DOLocationID': 143, 'trip_distance': 21.5},
 {'PULocationID': 249, 'DOLocationID': 230, 'trip_distance': 2.39},
 {'PULocationID': 100, 'DOLocationID': 113, 'trip_distance': 1.43},
 {'PULocationID': 161, 'DOLocationID': 239, 'trip_distance': 2.65},
 {'PULocationID': 137, 'DOLocationID': 141, 'trip_distance': 1.51},
 {'PULocationID': 186, 'DOLocationID': 170, 'trip_distance': 1.0},
 {'PULocationID': 234, 'DOLocationID': 231, 'trip_distanc

In [8]:
model_run_path = "/Users/dna/mlops-zoomcamp/chapter4/webservice/mlflow_artifacts/1/3fb7008b16534ce4b61ea373d96ddf42/artifacts/model"
# can use if mlflow server is down
# can also connect to aws or azure buckets
# logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.pyfunc.load_model(model_run_path)
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 3fb7008b16534ce4b61ea373d96ddf42

In [ ]:
preds = model.predict(dicts)

In [23]:

predictions = pd.DataFrame(preds)
predictions["ride_id"] = df["ride_id"]
predictions.columns = ["prediction", "ride_id"]
predictions["tpep_pickup_datetime"] = df["tpep_pickup_datetime"]
predictions["PULocationID"] = df["PULocationID"]
predictions["DOLocationID"] = df["DOLocationID"]
predictions["actual_duration"] = (
    (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
).map(lambda x: float(f"{x:.2f}"))
predictions["diff"] = predictions["prediction"] - predictions["actual_duration"]
predictions

,prediction,ride_id,tpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,diff
0,17.760214,f527a0d3-483d-422a-a0fd-253558622f88,2024-04-01 00:02:40,161,7,28.03,-10.269786
1,12.034739,8638a9ee-558d-413f-86a4-66e9c2a0f744,2024-04-01 00:41:12,264,264,14.28,-2.245261
2,13.947381,955357bd-7ce0-433f-916e-479de8b4c0f1,2024-04-01 00:48:42,186,236,16.80,-2.852619
3,15.939676,573376ae-b4e9-4c8d-baf5-1a9c10e9f888,2024-04-01 00:56:02,137,164,9.12,6.819676
4,12.588507,775c1146-b91d-4efd-b423-0863185c2fa8,2024-04-01 00:08:32,236,263,1.87,10.718507
...,...,...,...,...,...,...,...
3456844,16.426826,33b81650-d504-4599-ba3e-d01c981ad9c2,2024-04-22 21:43:31,239,158,17.93,-1.503174
3456845,19.344513,f53db255-7377-4095-9a59-957fc513b0a3,2024-04-22 21:28:44,234,249,6.77,12.574513
3456846,14.458734,3c777957-9bac-4ac7-a032-8a95c2880c15,2024-04-22 21:05:36,75,249,26.25,-11.791266
3456847,16.962938,b39a028d-19e5-4680-9507-ce1918bbc5b4,2024-04-22 21:35:40,161,170,9.68,7.282938


In [25]:
predictions.to_parquet("output/predictions.parquet")